# Adapted from tutorial nemo 3

In [1]:
from datetime import timedelta
import pathlib

from parcels import FieldSet, ParticleSet, JITParticle, AdvectionRK4_3D

from parcels import logger, XarrayDecodedFilter
logger.addFilter(XarrayDecodedFilter())  # Add a filter for the xarray decoding warning

enatl_data_path = pathlib.Path('enatl60_first_day')

u_files = sorted(enatl_data_path.glob('*1d_vozocrtx*'))
v_files = sorted(enatl_data_path.glob('*1d_vomecrty*'))
w_files = sorted(enatl_data_path.glob('*1d_vovecrtz*'))

assert len(u_files) == len(w_files)
assert len(u_files) == len(v_files)

enatl_mesh_mask_file = enatl_data_path / 'mesh_mask_eNATL60FARSHE_3.6.nc'

In [2]:
filenames = {
    'U': {'lon': enatl_mesh_mask_file, 'lat': enatl_mesh_mask_file, 'depth': enatl_mesh_mask_file, 'data': u_files},
    'V': {'lon': enatl_mesh_mask_file, 'lat': enatl_mesh_mask_file, 'depth': enatl_mesh_mask_file, 'data': v_files},
    'W': {'lon': enatl_mesh_mask_file, 'lat': enatl_mesh_mask_file, 'depth': enatl_mesh_mask_file, 'data': w_files},
}

variables = {
    'U': 'vozocrtx',
    'V': 'vomecrty',
    'W': 'vovecrtz',
}

dimensions = {
    'U': {'lon': 'glamf', 'lat': 'gphif', 'depth': 'gdepw_0', 'time': 'time_counter'},
    'V': {'lon': 'glamf', 'lat': 'gphif', 'depth': 'gdepw_0', 'time': 'time_counter'},
    'W': {'lon': 'glamf', 'lat': 'gphif', 'depth': 'gdepw_0', 'time': 'time_counter'},
}


In [3]:
# Will raise KeyError 'time_counter'
fieldset = FieldSet.from_nemo(filenames, variables, dimensions)

KeyError: 'time_counter'

In [4]:
# Try with depthw from w-file
filenames = {
    'U': {'lon': enatl_mesh_mask_file, 'lat': enatl_mesh_mask_file, 'depth': w_files[0], 'data': u_files},
    'V': {'lon': enatl_mesh_mask_file, 'lat': enatl_mesh_mask_file, 'depth': w_files[0], 'data': v_files},
    'W': {'lon': enatl_mesh_mask_file, 'lat': enatl_mesh_mask_file, 'depth': w_files[0], 'data': w_files},
}

variables = {
    'U': 'vozocrtx',
    'V': 'vomecrty',
    'W': 'vovecrtz',
}

dimensions = {
    'U': {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw', 'time': 'time_counter'},
    'V': {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw', 'time': 'time_counter'},
    'W': {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw', 'time': 'time_counter'},
}


In [5]:
# Will raise KeyError 'depthw'
fieldset_depthw_from_wfiles_zero = FieldSet.from_nemo(filenames, variables, dimensions)

KeyError: 'depthw'

## Comparison with OceanParcels Nemo tutorial


In [6]:
import xarray

parcels_nemo_3d_data_dir = pathlib.Path('parcels_examples/NemoNorthSeaORCA025-N006_data')

u_op = xarray.open_dataset(parcels_nemo_3d_data_dir / 'ORCA025-N06_20000104d05U.nc')
v_op = xarray.open_dataset(parcels_nemo_3d_data_dir / 'ORCA025-N06_20000104d05U.nc')
w_op = xarray.open_dataset(parcels_nemo_3d_data_dir / 'ORCA025-N06_20000104d05U.nc')


# eNATL files
u_en = xarray.open_dataset(enatl_data_path / 'eNATL60FARSHE-BLBT02_y2010m01d01.1d_vozocrtx_0-3882m.nc')
v_en = xarray.open_dataset(enatl_data_path / 'eNATL60FARSHE-BLBT02_y2010m01d01.1d_vomecrty_0-3882m.nc')

# derived from the files above using xgcm
#    https://xgcm.readthedocs.io/en/latest/xgcm-examples/04_nemo_idealized.html#3d-flow
# compared to one vovecrtz netcdf file exported from the eNATL60 model
#    eNATL60FARSHE-BLBT02_y2010m01d01.1h_vo_vecrtz_0-3882m  # the vo_ my change to not load it by mistake
w_en = xarray.open_dataset(enatl_data_path / 'eNATL60FARSHE-BLBT02_y2010m01d01.1d_vovecrtz_0-3882m.nc')



In [7]:
u_op

<xarray.Dataset>
Dimensions:               (depthu: 75, y: 201, x: 151, time_counter: 1,
                           axis_nbounds: 2)
Coordinates:
  * depthu                (depthu) float32 0.5058 1.556 ... 5.698e+03 5.902e+03
    nav_lat               (y, x) float32 ...
    nav_lon               (y, x) float32 ...
    time_centered         (time_counter) datetime64[ns] ...
  * time_counter          (time_counter) datetime64[ns] 2000-01-02T12:00:00
Dimensions without coordinates: y, x, axis_nbounds
Data variables:
    tauuo                 (time_counter, y, x) float32 ...
    time_centered_bounds  (time_counter, axis_nbounds) datetime64[ns] ...
    time_counter_bounds   (time_counter, axis_nbounds) datetime64[ns] ...
    uo                    (time_counter, depthu, y, x) float32 ...
    uos                   (time_counter, y, x) float32 ...
Attributes:
    name:         ORCA025-N006_5d_19961231_20011231
    description:  ocean U grid variables
    title:        ocean U grid variables
    Conventions:  CF-1.5
    production:   An IPSL model
    timeStamp:    2016-Aug-11 04:38:57 BST
    history:      Tue Mar 24 13:05:09 2020: ncks -d y,700,900 -d x,1050,1200 ...
    NCO:          netCDF Operators version 4.7.5 (Homepage = http://nco.sf.ne...

In [8]:
u_en

<xarray.Dataset>
Dimensions:       (y: 503, x: 593, depthu: 237, time_counter: 1)
Coordinates:
  * depthu        (depthu) float32 0.4805 1.559 2.794 ... 3.857e+03 3.883e+03
  * time_counter  (time_counter) datetime64[ns] 2010-01-01T12:00:00
Dimensions without coordinates: y, x
Data variables:
    nav_lon       (y, x) float32 ...
    nav_lat       (y, x) float32 ...
    vozocrtx      (time_counter, depthu, y, x) float32 ...
Attributes:
    start_date:        -1
    output_frequency:  N/A
    CONFIG:            N/A
    CASE:              N/A

## Differences
OP dataset: nav_lon/lat are coords
OP dataset: axis_nbounds: 2

In [9]:
import glob
import cftime
# Coordinates/mesh
data_path = str(parcels_nemo_3d_data_dir) + '/'
ufiles = sorted(glob.glob(data_path + 'ORCA*U.nc'))
vfiles = sorted(glob.glob(data_path + 'ORCA*V.nc'))
wfiles = sorted(glob.glob(data_path + 'ORCA*W.nc'))
mesh_mask = data_path + 'coordinates.nc'

op_mesh_mask = xarray.open_dataset(mesh_mask, decode_times=False)
enatl_mesh_mask = xarray.open_dataset(enatl_mesh_mask_file)

In [10]:
op_mesh_mask

<xarray.Dataset>
Dimensions:     (time: 1, y: 201, x: 151)
Coordinates:
  * time        (time) float32 0.0
Dimensions without coordinates: y, x
Data variables: (12/20)
    e1f         (time, y, x) float64 ...
    e1t         (time, y, x) float64 ...
    e1u         (time, y, x) float64 ...
    e1v         (time, y, x) float64 ...
    e2f         (time, y, x) float64 ...
    e2t         (time, y, x) float64 ...
    ...          ...
    gphiu       (time, y, x) float64 ...
    gphiv       (time, y, x) float64 ...
    nav_lat     (y, x) float32 ...
    nav_lev     float32 ...
    nav_lon     (y, x) float32 ...
    time_steps  (time) int32 ...
Attributes:
    Conventions:  GDT 1.2
    file_name:    coordinates_ORCA_R025.nc
    TimeStamp:    2000-APR-18 13:48:20 GMT-0200
    history:      Thu Jul 19 16:14:39 2018: ncks -d y,700,900 -d x,1050,1200 ...
    NCO:          4.4.4

In [11]:
enatl_mesh_mask

<xarray.Dataset>
Dimensions:       (t: 1, y: 503, x: 593, z: 237)
Dimensions without coordinates: t, y, x, z
Data variables: (12/44)
    e1f           (t, y, x) float64 ...
    e1t           (t, y, x) float64 ...
    e1u           (t, y, x) float64 ...
    e1v           (t, y, x) float64 ...
    e2f           (t, y, x) float64 ...
    e2t           (t, y, x) float64 ...
    ...            ...
    tmask         (t, z, y, x) int8 ...
    tmaskutil     (t, y, x) int8 ...
    umask         (t, z, y, x) int8 ...
    umaskutil     (t, y, x) int8 ...
    vmask         (t, z, y, x) int8 ...
    vmaskutil     (t, y, x) int8 ...
Attributes:
    file_name:  mesh_mask.nc
    TimeStamp:  13/07/2018 22:32:53 +0200
    history:    Mon Oct 17 16:49:09 2022: ncks -O -F -d x,5220,5812 -d y,4048...
    NCO:        4.4.6